In [ ]:
import os

In [ ]:
os.chdir("/content/fsl")

In [ ]:
# import packages
from tqdm import tqdm

import torch as th
from torch import nn
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torch.nn.modules.loss import CrossEntropyLoss

from torchvision.transforms import Compose, Grayscale, CenterCrop, ToTensor, ToPILImage, Resize
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.datasets import ImageFolder

from src import Learner, TaskSampler, PrototypicalNetworks

In [ ]:
# configurations
CONFIG = {}
CONFIG['shot'] = 5
CONFIG['way'] = 5
CONFIG['query'] = 10
DEVICE = 'cuda' if th.cuda.is_available() else 'cpu'
if DEVICE is 'cuda':
  print ('cuda is available as device')

cuda is available as device


In [ ]:
# Image transformation pipeline

image_transform_pipe = Compose(
    [
        Grayscale(num_output_channels=3),
        CenterCrop(224),
        Resize(128),
        ToTensor()
    ]
)

# load Train dataset
train_dataset = ImageFolder(root = "./few_shot_data/train", transform=image_transform_pipe)
# load Validation dataset
val_dataset = ImageFolder(root = "./few_shot_data/val", transform=image_transform_pipe)
# load Test dataset
test_dataset = ImageFolder(root ="./few_shot_data/test", transform=image_transform_pipe)

In [ ]:
# Create task samplers
train_sampler = TaskSampler(train_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=500)

validation_sampler = TaskSampler(val_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=100)

test_sampler = TaskSampler(test_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=20)

In [ ]:
# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_sampler=train_sampler,
    num_workers=8,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_sampler=validation_sampler,
    num_workers=8,
    pin_memory=True,
    collate_fn=validation_sampler.episodic_collate_fn,
)

test_loader = DataLoader(
    test_dataset,
    batch_sampler=test_sampler,
    num_workers=8,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

## Training first Prototypical Networks with SGD Optimizer

In [ ]:
# load pretrained model for backbone
pretrained_weights = ResNet18_Weights.IMAGENET1K_V1
backbone_model_1 = resnet18(weights=pretrained_weights)

# Change final layer output from number of classes 
# to flattened feature vector from resnet's preceding architecure
backbone_model_1.fc = nn.Flatten()
model_1 = backbone_model_1.cuda()

# Create FSL classifier
proton_classifier_1 = PrototypicalNetworks(backbone=model_1, output_softmax_score=False)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
learning_rate = 0.05
optimizer_1 = SGD(proton_classifier_1.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()

proton_c1_best_val_accuracy = Learner.fit(
    train_data_loader = train_loader,
    val_data_loader = val_loader,
    model = proton_classifier_1,
    optimizer=optimizer_1, 
    loss_function = loss_function,
    epochs = 100,
    model_path="./models/protonet100.pkl",
    tensorboard_log_path = "./tensorboard/protonet/"
  )

Training Epoch 0


100%|██████████| 500/500 [00:28<00:00, 17.55it/s, episode_accuracy=0.96, episode_loss=0.122, epoch_accuracy=0.886, epoch_loss=0.32]


Validating Epoch 0


100%|██████████| 100/100 [00:04<00:00, 21.19it/s, episode_accuracy=0.9, overall_accuracy=0.859]


Training Epoch 1


...

100%|██████████| 100/100 [00:04<00:00, 20.43it/s, episode_accuracy=0.92, overall_accuracy=0.933]


Training Epoch 99


100%|██████████| 500/500 [00:27<00:00, 18.31it/s, episode_accuracy=1, episode_loss=0.00158, epoch_accuracy=0.998, epoch_loss=0.0081]


Validating Epoch 99


100%|██████████| 100/100 [00:04<00:00, 20.19it/s, episode_accuracy=0.9, overall_accuracy=0.932]


In [ ]:
# Prototypical classifier 1 best validation accuracy
proton_c1_best_val_accuracy

0.9448

## Training second Prototypical Networks with Adam Optimizer

In [ ]:
# load pretrained backbone
pretrained_weights_2 = ResNet18_Weights.IMAGENET1K_V1
backbone_model_2 = resnet18(weights=pretrained_weights)

# Change final layer output from number of classes 
# to flattened feature vector from resnet's preceding architecure
backbone_model_2.fc = nn.Flatten()
model_2 = backbone_model_2.cuda()

# load FSL classifer
proton_classifier_2 = PrototypicalNetworks(backbone=model_2, output_softmax_score=False)

In [ ]:
learning_rate = 0.05
optimizer_2 = Adam(proton_classifier_2.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()

proton_c2_best_val_accuracy = Learner.fit(
    train_data_loader = train_loader,
    val_data_loader = val_loader,
    model = proton_classifier_2,
    optimizer=optimizer_2, 
    loss_function = loss_function,
    epochs = 100,
    model_path="./models/protonet100_2.pkl",
    tensorboard_log_path = "./tensorboard/protonet/"
  )

Training Epoch 0


100%|██████████| 500/500 [00:28<00:00, 17.66it/s, episode_accuracy=0.44, episode_loss=1.31, epoch_accuracy=0.408, epoch_loss=1.43]


Validating Epoch 0


100%|██████████| 100/100 [00:04<00:00, 20.28it/s, episode_accuracy=0.38, overall_accuracy=0.464]


Training Epoch 1


...

100%|██████████| 100/100 [00:04<00:00, 20.46it/s, episode_accuracy=0.98, overall_accuracy=0.825]


Training Epoch 99


100%|██████████| 500/500 [00:27<00:00, 18.03it/s, episode_accuracy=0.98, episode_loss=0.0319, epoch_accuracy=0.975, epoch_loss=0.0705]


Validating Epoch 99


100%|██████████| 100/100 [00:05<00:00, 19.63it/s, episode_accuracy=0.74, overall_accuracy=0.809]


In [ ]:
# Prototypical classifier 2 best validation accuracy
proton_c2_best_val_accuracy

0.8248

## Get test accuracy for each classifier

In [ ]:
# Get test scores of Prototypical Networks classifier 1

correct_prediction_count = 0
total_prediction_count = 0
for (support_images, support_labels, query_images, query_labels, _) in test_loader:
  prediction_labels = Learner.predict_images(
      support_images.to(DEVICE),
      support_labels.to(DEVICE),
      query_images.to(DEVICE),
      proton_classifier_1
    )
  correct_prediction_count += (prediction_labels==query_labels).sum().item()
  total_prediction_count += len(prediction_labels)

overall_test_accuracy  = correct_prediction_count/total_prediction_count
print ("Prototypical Networks Classifier 1 test accuracy: ", overall_test_accuracy)

Prototypical Networks Classifier 1 test accuracy:  0.941


In [ ]:
# Get test scores of Prototypical Networks classifier 2

correct_prediction_count = 0
total_prediction_count = 0
for (support_images, support_labels, query_images, query_labels, _) in test_loader:
  prediction_labels = Learner.predict_images(
      support_images.to(DEVICE),
      support_labels.to(DEVICE),
      query_images.to(DEVICE),
      proton_classifier_2
    )
  correct_prediction_count += (prediction_labels==query_labels).sum().item()
  total_prediction_count += len(prediction_labels)

overall_test_accuracy  = correct_prediction_count/total_prediction_count
print ("Prototypical Networks Classifier 2 test accuracy: ", overall_test_accuracy)

Prototypical Networks Classifier 2 test accuracy:  0.779
